In [27]:
import os
import pandas as pd
import numpy as np

# Create Spark context
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName('dj').setMaster('local[4]')
sc = SparkContext(conf=conf)

# Create Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dj').getOrCreate()
# Print spark
sc.applicationId

'local-1698516398527'

In [2]:
# Don't change this file path
file_path = "airports.csv"
# Read in the airports data
airports = spark.read.csv(file_path, header=True)
# Don't change this file path
file_path = "./flights_small.csv"
# Read in the airports data
flights = spark.read.csv(file_path, header=True)
# Add spark_temp to the catalog
flights.createOrReplaceTempView('flights')
# Add spark_temp to the catalog
airports.createOrReplaceTempView('airports')

In [3]:
# Create the DataFrame flights
flights = spark.table('flights')

# Show the head
flights.show()

# Add duration_hrs
flights = flights.withColumn('duration_hrs',flights.air_time/60)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [4]:
# Don't change this query
query = "SELECT * FROM flights LIMIT 10"

# Get the first 10 rows of flights
flights10 = spark.sql(query)

# Show the results
flights10.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [5]:
spark.sql("SELECT dest, tailnum FROM flights WHERE air_time > 10 limit 10").show()

+----+-------+
|dest|tailnum|
+----+-------+
| LAX| N846VA|
| HNL| N559AS|
| SFO| N847VA|
| SJC| N360SW|
| BUR| N612AS|
| DEN| N646SW|
| OAK| N422WN|
| SFO| N361VA|
| SAN| N309AS|
| ORD| N564AS|
+----+-------+



In [6]:
spark.sql("SELECT round(AVG(air_time) / 60,4) as avg_air_time FROM flights GROUP BY origin, carrier;").show(3,truncate=False)

+------------+
|avg_air_time|
+------------+
|2.6411      |
|3.4816      |
|2.5649      |
+------------+
only showing top 3 rows



In [7]:
spark.sql("SELECT origin, dest, COUNT(*) as num_flights FROM flights GROUP BY origin, dest ORDER BY 3 DESC;").show(5,truncate=False)

+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|SEA   |SFO |482        |
|SEA   |LAX |450        |
|SEA   |ANC |380        |
|SEA   |LAS |364        |
|SEA   |DEN |351        |
+------+----+-----------+
only showing top 5 rows



In [ ]:
# SQL in a nutshell
# As you move forward, it will help to have a basic understanding of SQL. A more in depth look can be found here.

# A SQL query returns a table derived from one or more tables contained in a database.

# Every SQL query is made up of commands that tell the database what you want to do with the data. The two commands that every query has to contain are SELECT and FROM.

# The SELECT command is followed by the columns you want in the resulting table.

# The FROM command is followed by the name of the table that contains those columns. The minimal SQL query is:

# SELECT * FROM my_table;
# The * selects all columns, so this returns the entire table named my_table.

# Similar to .withColumn(), you can do column-wise computations within a SELECT statement. For example,

# SELECT origin, dest, air_time / 60 FROM flights;
# returns a table with the origin, destination, and duration in hours for each flight.

# Another commonly used command is WHERE. This command filters the rows of the table based on some logical condition you specify. The resulting table contains the rows where your condition is true. For example, if you had a table of students and grades you could do:

# SELECT * FROM students
# WHERE grade = 'A';
# to select all the columns and the rows containing information about students who got As.

# Which of the following queries returns a table of tail numbers and destinations for flights that lasted more than 10 hours?


In [ ]:
# SQL in a nutshell (2)
# Another common database task is aggregation. That is, reducing your data by breaking it into chunks and summarizing each chunk.

# This is done in SQL using the GROUP BY command. This command breaks your data into groups and applies a function from your SELECT statement to each group.

# For example, if you wanted to count the number of flights from each of two origin destinations, you could use the query

# SELECT COUNT(*) FROM flights
# GROUP BY origin;
# GROUP BY origin tells SQL that you want the output to have a row for each unique value of the origin column. The SELECT statement selects the values you want to populate each of the columns. Here, we want to COUNT() every row in each of the groups.

# It's possible to GROUP BY more than one column. When you do this, the resulting table has a row for every combination of the unique values in each column. The following query counts the number of flights from SEA and PDX to every destination airport:

# SELECT origin, dest, COUNT(*) FROM flights
# GROUP BY origin, dest;
# The output will have a row for every combination of the values in origin and dest (i.e. a row listing each origin and destination that a flight flew to). There will also be a column with the COUNT() of all the rows in each group.

# Remember, a more in depth look at SQL can be found here.

# What information would this query get? Remember the flights table holds information about flights that departed PDX and SEA in 2014 and 2015. Note that AVG() function gets the average value of a column!

# SELECT AVG(air_time) / 60 FROM flights
# GROUP BY origin, carrier;


In [8]:
# Filter flights by passing a string
long_flights1 = flights.filter("distance > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.distance > 1000)

# Print the data to check they're equal
long_flights1.show(3,truncate=False)
long_flights2.show(3,truncate=False)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |6.0         |
|2014|4    |19 |1236    |-4       |1508    |-7       |AS     |N309AS |490   |SEA   |SAN |135     |1050    |12  |36    |2.25        |
|2014|11   |19 |1812    |-3       |2352    |-4       |AS     |N564AS |26    |SEA   |ORD |198     |1721    |18  |12    |3.3         |
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 3 rows

+----+-----+---+--------+---------+--------+

In [9]:
flights.filter("air_time > 120").show(3,truncate=False)
flights.filter(flights.air_time > 120).show(3,truncate=False)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs      |
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |2.2               |
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |6.0               |
|2014|3    |9  |754     |-1       |1015    |1        |AS     |N612AS |522   |SEA   |BUR |127     |937     |7   |54    |2.1166666666666667|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
only showing top 3 rows

+-

In [10]:
# Select the first set of columns
selected1 = flights.select('tailnum', 'origin', 'dest')

# Select the second set of columns
temp = flights.select(flights.origin, flights.dest, flights.carrier)

# Define first filter
filterA = flights.origin == "SEA"

# Define second filter
filterB = flights.dest == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp\
    .filter(filterA)\
    .filter(filterB)
selected1.show(3,truncate=False)
selected2.show(3,truncate=False)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
|N846VA |SEA   |LAX |
|N559AS |SEA   |HNL |
|N847VA |SEA   |SFO |
+-------+------+----+
only showing top 3 rows

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|SEA   |PDX |OO     |
|SEA   |PDX |OO     |
|SEA   |PDX |OO     |
+------+----+-------+
only showing top 3 rows



In [11]:
# Define avg_speed
avg_speed = (flights.distance/(flights.air_time/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select("origin", "dest", "tailnum", avg_speed)

# Create the same table using a SQL expression
speed2 = flights\
    .selectExpr(
        "origin",
        "dest",
        "tailnum",
        "distance/(air_time/60) as avg_speed"
    )
speed1.show(3,truncate=False)
speed2.show(3,truncate=False)

+------+----+-------+------------------+
|origin|dest|tailnum|avg_speed         |
+------+----+-------+------------------+
|SEA   |LAX |N846VA |433.6363636363636 |
|SEA   |HNL |N559AS |446.1666666666667 |
|SEA   |SFO |N847VA |367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows

+------+----+-------+------------------+
|origin|dest|tailnum|avg_speed         |
+------+----+-------+------------------+
|SEA   |LAX |N846VA |433.6363636363636 |
|SEA   |HNL |N559AS |446.1666666666667 |
|SEA   |SFO |N847VA |367.02702702702703|
+------+----+-------+------------------+
only showing top 3 rows



In [12]:
flights.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [13]:
flights.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)



In [14]:
from pyspark.sql.types import IntegerType
    
flights = flights.withColumn("distance", flights["distance"].cast(IntegerType()))
flights = flights.withColumn("air_time", flights["air_time"].cast(IntegerType()))
flights = flights.withColumn("dep_delay", flights["dep_delay"].cast(IntegerType()))

In [15]:
flights\
    .filter(flights.origin == "PDX")\
	.groupBy()\
	.min("distance")\
	.show()

# Find the longest flight from SEA in terms of air time
flights\
    .filter(flights.origin == "SEA")\
	.groupBy()\
	.max("air_time")\
	.show()

+-------------+
|min(distance)|
+-------------+
|          106|
+-------------+

+-------------+
|max(air_time)|
+-------------+
|          409|
+-------------+



In [16]:
# Average duration of Delta flights
flights\
    .filter(flights.carrier == "DL")\
    .filter(flights.origin == "SEA")\
    .groupBy()\
    .avg("air_time")\
    .show(3,truncate=False)

# Total hours in the air
flights\
    .withColumn("duration_hrs", flights.air_time/60)\
    .groupBy()\
    .sum("duration_hrs")\
    .show(3,truncate=False)# Average duration of Delta flights


+------------------+
|avg(air_time)     |
+------------------+
|188.20689655172413|
+------------------+

+------------------+
|sum(duration_hrs) |
+------------------+
|25289.600000000126|
+------------------+



In [17]:

# Group by tailnum
by_plane = flights.groupBy("tailnum")

# Number of flights each plane made
by_plane.count().show(3,truncate=False)

# Group by origin
by_origin = flights.groupBy("origin")

# Average duration of flights from PDX and SEA
by_origin.avg("air_time").show(3,truncate=False)

+-------+-----+
|tailnum|count|
+-------+-----+
|N442AS |38   |
|N102UW |2    |
|N36472 |4    |
+-------+-----+
only showing top 3 rows

+------+------------------+
|origin|avg(air_time)     |
+------+------------------+
|SEA   |160.4361496051259 |
|PDX   |137.11543248288737|
+------+------------------+



In [18]:
# Import pyspark.sql.functions as F
import pyspark.sql.functions as F

# Group by month and dest
by_month_dest = flights.groupBy('month', 'dest')

# Average departure delay by month and destination
by_month_dest.avg('dep_delay').show(3,truncate=False)

# Standard deviation of departure delay
by_month_dest.agg(F.stddev('dep_delay')).show(3,truncate=False)

+-----+----+-------------------+
|month|dest|avg(dep_delay)     |
+-----+----+-------------------+
|11   |TUS |-2.3333333333333335|
|11   |ANC |7.529411764705882  |
|1    |BUR |-1.45              |
+-----+----+-------------------+
only showing top 3 rows

+-----+----+------------------+
|month|dest|stddev(dep_delay) |
+-----+----+------------------+
|11   |TUS |3.0550504633038935|
|11   |ANC |18.604716401245316|
|1    |BUR |15.22627576540667 |
+-----+----+------------------+
only showing top 3 rows



In [ ]:
# Joining
# Another very common data operation is the join. Joins are a whole topic unto themselves, so in this course we'll just look at simple joins. If you'd like to learn more about joins, you can take a look here.

# A join will combine two different tables along a column that they share. This column is called the key. Examples of keys here include the tailnum and carrier columns from the flights table.

# For example, suppose that you want to know more information about the plane that flew a flight than just the tail number. This information isn't in the flights table because the same plane flies many different flights over the course of two years, so including this information in every row would result in a lot of duplication. To avoid this, you'd have a second table that has only one row for each plane and whose columns list all the information about the plane, including its tail number. You could call this table planes

# When you join the flights table to this table of airplane information, you're adding all the columns from the planes table to the flights table. To fill these columns with information, you'll look at the tail number from the flights table and find the matching one in the planes table, and then use that row to fill out all the new columns.

# Now you'll have a much bigger table than before, but now every row has all information about the plane that flew that flight!

In [19]:
# Examine the data
print(airports.show(3,truncate=False))

# Rename the faa column
airports = airports.withColumnRenamed("faa", "dest")

# Join the DataFrames
flights_with_airports = flights.join(airports, on='dest', how='leftouter')

# Examine the new DataFrame
print(flights_with_airports.show(3,truncate=False))

+---+-----------------------------+----------+-----------+----+---+---+
|faa|name                         |lat       |lon        |alt |tz |dst|
+---+-----------------------------+----------+-----------+----+---+---+
|04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |
|06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |
|06C|Schaumburg Regional          |41.9893408|-88.1012428|801 |-6 |A  |
+---+-----------------------------+----------+-----------+----+---+---+
only showing top 3 rows

None
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------+------------------+---------+-----------+---+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|duration_hrs|name              |lat      |lon        |alt|tz |dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+-----